<a href="https://colab.research.google.com/github/ahmadhajmosa/Machine-learning-labs/blob/AlessandroFornasier/Session_3_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab on Machine Learning and Applications in Intelligent Vehicles
## Session 3: Keras

### * Intro into Keras

### * CNN using Keras

### * LSTM using Keras

### * Variational Auto encoder using Keras

### * Seq2Seq Model with keras


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# Sequential define that we append layer in a sequential way
model = Sequential()

# Append dense (fully connected) layer with an array of shape (*,16) as input
# and an array of shape (32,*) as output where the ] means that we can
# have as much samples as we want of size 16 for the input 32 for the output
model.add(Dense(32, input_shape = (16,), name = "inputLayer"))

# Append other hidden layers, pay attention that after the first layer is not
# needed to specify the size of the input
model.add(Dense(100, activation = "relu", name = "hiddenLayer1"))
model.add(Dropout(0.1))
model.add(Dense(200, activation = "sigmoid", name = "hiddenLayer2"))
model.add(Dropout(0.1))
model.add(Dense(30, name = "hiddenLayer3"))
model.add(Dense(100, name = "hiddenLayer4"))

# Append the output layer
model.add(Dense(32, name = "outputLayer"))

# Model visualization 
SVG(model_to_dot(model).create(prog='dot', format='svg'))


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# Sequential define that we append layer in a sequential way
model = Sequential()

# Append convolutional layer where the input is an RGB image of size 32x32
# kernel size of 3 (3,3) and 64 output/filters
model.add(Conv2D(64, (3,3), input_shape = (3,32,32), padding = "same"))

# Append other hidden layers, pay attention that after the first layer is not
# needed to specify the size of the input
model.add(Conv2D(32, (3,3), padding = "same"))

# Append the flatten layer
model.add(Flatten())

# Model visualization
display(model.summary())
print('\n')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# the to_categorical method converts labels to one hot encoding format
# labels = [1,2,1,1] --> [[0,1] [1,0] [0,1] [0,1]]
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model definition
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop (binary crossentropy 
# if we have only 2 classes) otherwise we use categorical
# crossentropy for classification problems
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Data scaling from [0,255] to [0,1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Image augmentation, if i want to increase the universality of my NN
# i can modify some parameters of he image like size, contrast, illumination,
# crop it, rotate it, etc..

if not data_augmentation:
    print('Not using data augmentation.')
    # Train the model
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(x_test, y_test), workers=4)

# Show the summary
model.summary()

In [0]:
# Functional modelling is more flexible than Sequential modelling
# it is more low level and allow us to manipulate more the model

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
import numpy as np

# ---------------------------- NN1 ----------------------------

# This returns a tensor
inputs1 = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x1 = Dense(64, activation='relu')(inputs1)
x1 = Dense(64, activation='relu')(x1)

# -------------------------------------------------------------

# ---------------------------- NN2 ----------------------------

# This returns a tensor
inputs2 = Input(shape=(10,))

# a layer instance is callable on a tensor, and returns a tensor
x2 = Dense(64, activation='relu')(inputs2)
x2 = Dense(64, activation='relu')(x2)

# -------------------------------------------------------------

# Merge the two output of the 2 NNs by concatenation [x1,x2],
# we can also merge the outputs of the NNs by averaging, etc...
merged = Concatenate()([x1,x2])

# ---------------------------- NN3 ----------------------------

# a layer instance is callable on a tensor, and returns a tensor
x3 = Dense(64, activation='relu')(merged)
x3 = Dense(64, activation='relu')(x3)

y1 = Dense(64, activation='relu', name = "output1")(x3)
y2 = Dense(64, activation='softmax', name = "output2")(x3)

# -------------------------------------------------------------

# Cost function definitions for the two outputs
losses = dict()
losses['output1'] = 'categorical_crossentropy'
losses['output2'] = 'mse'

# Metrics definitions for the two outputs
metrics = dict()
metrics['output1'] = 'acc'
metrics['output2'] = 'mse'

# Model definition
model = Model([inputs1, inputs2], [y1,y2])

# Model traning
model.compile(optimizer = "adam", loss = losses, metrics = metrics)

# Model visualization
display(model.summary())
print('\n')
SVG(model_to_dot(model).create(prog='dot', format='svg'))